In [34]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [35]:
# Load all file ends with .md
loader = DirectoryLoader('data/', glob="*.md", show_progress=True, loader_cls=UnstructuredMarkdownLoader)
# Break documents into smaller chunks, RecursiveCharacterTextSplitter with chunk_size = 2000 is good
docs = loader.load_and_split(RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 100))
# Embedded to local data vector database
db  = Chroma.from_documents(docs, OllamaEmbeddings(model="mxbai-embed-large",show_progress=True))

OllamaEmbeddings: 100%|██████████| 8/8 [00:35<00:00,  4.46s/it]


In [36]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4}) # Turn VectorStore into a Retriever
local_model = "llama3" # [Phi 3, Mistral, Gamma]
llm = ChatOllama(model=local_model, temperature = 0) # temperature > 1 more creative, random, temperature < 1 deterministic, repetitive

# System prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. 
    Keep the answer concise <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question} 
    Context: {context} 
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "context"],
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [46]:
question = "How can I run a job on Polaris? Write an example job submission script?"
print(question)
print(rag_chain.invoke(question))

How can I run a job on Polaris? Write an example job submission script?


OllamaEmbeddings: 100%|██████████| 1/1 [00:51<00:00, 51.29s/it]


To run a job on Polaris, you can use the `qsub` command to submit an interactive or batch job. For example, to submit an interactive job that requests one node for a period of one hour in the debug queue, requiring access to the `/home` and `eagle` filesystems, you would use the following command:

```
qsub -I -l select=1 -l filesystems=home:eagle -l walltime=1:00:00 -q debug -A <project_name>
```

This will request a node for one hour in the debug queue and provide access to the `/home` and `eagle` filesystems. After waiting in the queue, a shell prompt on a compute node will appear, allowing you to start building applications and testing GPU affinity scripts.

As for running multiple MPI applications on a node, you can use the script provided:

```
gpu=$((${num_gpus} - 1 - ${PMI_LOCAL_RANK} % ${num_gpus}))
export CUDA_VISIBLE_DEVICES=$gpu
echo “RANK= ${PMI_RANK} LOCAL_RANK= ${PMI_LOCAL_RANK} gpu= ${gpu}”
exec "$@"
```

This script can be placed just before the executable in the `mpie

In [38]:
question = "How to connect to polaris"
print(question)
print(rag_chain.invoke(question))

How to connect to polaris


OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.46s/it]


To connect to Polaris, you can log in using the following command:

ssh <username>@polaris.alcf.anl.gov
Then, type in the password from your CRYPTOCard/MobilePASS+ token.


In [40]:
question = "How many queues are there?"
print(question)
print(rag_chain.invoke(question))

How many queues are there?


OllamaEmbeddings: 100%|██████████| 1/1 [00:56<00:00, 56.42s/it]


There are five production queues you can target in your qsub (-q <queue name>):

1. debug
2. debug-scaling
3. prod
4. preemptable
5. demand


In [42]:
question = "Give me more details on these five production queues"
print(question)
print(rag_chain.invoke(question))

Give me more details on these five production queues


OllamaEmbeddings: 100%|██████████| 1/1 [00:28<00:00, 28.86s/it]


I don't know the answer to that question. The provided context does not mention five production queues, but rather provides general information about using the Polaris system, including logging in, accessing additional software, submitting and running jobs, and controlling where jobs run. If you're looking for specific information on production queues, I would recommend searching elsewhere or reaching out to support@alcf.anl.gov for assistance.


In [45]:
question = "What are the time max and node max of these five production queues"
print(question)
print(rag_chain.invoke(question))

What are the time max and node max of these five production queues


OllamaEmbeddings: 100%|██████████| 1/1 [00:01<00:00,  1.19s/it]


It looks like you're trying to get started on the Polaris system! That's a great step.

To answer your question, it seems like you're looking for information on how to control where your job runs on Polaris. Here are some key points:

* To run a job on specific nodes, use the `-l select` option followed by the node names, separated by `+`. For example: `-l select=1:vnode=<node name1>+1:vnode=<node name2>...`
* If you want to control the location of a few nodes but don't care about the rest, use the same syntax as above, but specify the number of nodes and the system (e.g., `system=foo`).
* To group nodes by rack or dragonfly group, use the `-l select` option with the `group` specifier. For example: `-l select=8:system=foo,place=scatter:group=tier0`
* If you want to run a job on specific racks or dragonfly groups, replace `tier0` with `tier1`.

Additionally, here are some general tips for getting started on Polaris:

* To log in, use the command `ssh <username>@polaris.alcf.anl.gov` and